In [137]:
# Encode the CSV as utf-8 for python
import csv

path =  './holdout_set.csv'

with open(path, 'r', encoding='utf-8', errors='ignore') as infile, open('holdout_set_encoded.csv', 'w') as outfile:
     inputs = csv.reader(infile)
     output = csv.writer(outfile)

     for index, row in enumerate(inputs):
         output.writerow(row)

In [147]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from scipy.stats import zscore
import math

In [152]:
import nltk
# nltk.download()
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
# MacOS garbage
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
# Pickle
X.to_pickle("./cleaned_data.pkl")

In [128]:
# Load Pickle
X = pd.read_pickle("./cleaned_data.pkl")

In [140]:
train_df = pd.read_csv('./holdout_set_encoded.csv')
train_df.head()

,Engagements,Followers at Posting,Created,Type,Description
0,NaN,36984682,2019-05-22 00:31:55 EDT,Photo,The @raptors even up the Eastern Conference Fi...
1,NaN,36984682,2019-05-21 23:20:41 EDT,Photo,The @raptors even the Eastern Conference Final...
2,NaN,36984682,2019-05-21 21:18:51 EDT,Video,@sergeibaka keeps it alive for the @raptors! (...
3,NaN,36955156,2019-05-21 18:20:09 EDT,Video,???? The SUPERHUMAN HANDS of Giannis & Kawhi ?...
4,NaN,36955156,2019-05-21 15:04:31 EDT,Video,?? the @raptors & @bucks in transition! #NBABr...


In [141]:
# train_sample_fraction = 0.2
# y = train_df[['Engagements']]
# X = train_df.loc[:, train_df.columns != 'Engagements']
X = train_df[['Followers at Posting']]
# X.dtypes

In [142]:
rows = train_df.size
X['Year'] = pd.DataFrame(0, index=range(rows), columns=range(1))
X['Month'] = pd.DataFrame(0, index=range(rows), columns=range(1))
X['Day'] = pd.DataFrame(0, index=range(rows), columns=range(1))
X['Hour'] = pd.DataFrame(0, index=range(rows), columns=range(1))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [143]:
dates = train_df[['Created']]

for r in range(dates.values.size):
    row = dates.values[r].tolist()
    row = "".join(row).split()
    date = row[0].split('-')
    year = date[0]
    month = date[1]
    day = date[2]
    hour = row[1].split(":")[0]
    X['Year'][r] = year
    X['Month'][r] = month
    X['Day'][r] = day
    X['Hour'][r] = hour

X.head()

,Followers at Posting,Year,Month,Day,Hour
0,36984682,2019,5,22,0
1,36984682,2019,5,21,23
2,36984682,2019,5,21,21
3,36955156,2019,5,21,18
4,36955156,2019,5,21,15


In [144]:
Media_Type = train_df[['Type']]

label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(Media_Type)
Media_Type = label_encoder.transform(Media_Type)
# print([x for x in Media_Type])
label_encoder.classes_

X['Type'] = Media_Type
X.tail()

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,Followers at Posting,Year,Month,Day,Hour,Type
995,24893189,2017,10,3,21,2
996,24893189,2017,10,3,7,1
997,24885537,2017,10,2,20,2
998,24885537,2017,10,2,17,2
999,24875542,2017,10,1,23,2


In [145]:
rows = train_df.size
X['neg'] = pd.DataFrame(0.0, index=range(rows), columns=range(1))
X['neu'] = pd.DataFrame(0.0, index=range(rows), columns=range(1))
X['pos'] = pd.DataFrame(0.0, index=range(rows), columns=range(1))
X['compound'] = pd.DataFrame(0.0, index=range(rows), columns=range(1))

In [154]:
# Sentiment analysis on "Description" feature
description = train_df[['Description']]

analyser = SentimentIntensityAnalyzer()

for r in range(description.values.size):
    row = description.values[r].tolist()
    if isinstance(row[0], str):
        snt = analyser.polarity_scores(row[0])
        X['neg'][r] = snt['neg']
        X['neu'][r] = snt['neu']
        X['pos'][r] = snt['pos']
        X['compound'][r] = snt['compound']

X.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/anaconda3/lib/python3.7/si

,Followers at Posting,Year,Month,Day,Hour,Type,neg,neu,pos,compound
0,36984682,2019,5,22,0,1,0.0,1.000,0.000,0.0000
1,36984682,2019,5,21,23,1,0.0,1.000,0.000,0.0000
2,36984682,2019,5,21,21,2,0.0,0.711,0.289,0.5027
3,36955156,2019,5,21,18,2,0.0,1.000,0.000,0.0000
4,36955156,2019,5,21,15,2,0.0,1.000,0.000,0.0000


In [130]:
X_train, X_test, y_train, y_test = train_test_split(X, y ,test_size=train_sample_fraction, random_state=420)

xgb = XGBRegressor(
#     eta=0.3,
#     gamma=0.05,
#     max_depth=10,
#     min_child_weight=0.5,
#     max_delta_step=1,
#     reg_lambda=1,
#     alpha=1,
#     objective='reg:linear'
#     eval_metric='mae'
#     colsample_bytree=0.4,
#     gamma=0,                 
#     learning_rate=0.07,
#     max_depth=3,
#     min_child_weight=1.5,
#     n_estimators=10000,                                                                    
#     reg_alpha=0.75,
#     reg_lambda=0.45,
#     subsample=0.6,
)

In [131]:
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)

In [132]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    sum = 0
    for t, p in zip(y_true, y_pred):
        sum += abs((t - p)/t)
    sum /= len(y_true)
    
    return sum[0]*100

In [139]:
MAPE = mean_absolute_percentage_error(y_test, y_pred)
print(MAPE)
MAE = mean_absolute_error(y_test, y_pred)
print(MAE)

9.864421673526314
57095.0838662323


In [25]:
X_norm = (X-X.mean())/X.std()
X_norm

,Followers at Posting,Year,Month,Day,Hour,Type,neg,neu,pos,compound
0,1.749016,1.344857,-0.254866,0.649661,0.966564,0.599279,-0.322659,0.700588,-0.581794,-0.332373
1,1.749016,1.344857,-0.254866,0.649661,0.829939,0.599279,-0.322659,0.700588,-0.581794,-0.332373
2,1.749016,1.344857,-0.254866,0.649661,0.829939,0.599279,-0.322659,0.700588,-0.581794,-0.332373
3,1.749016,1.344857,-0.254866,0.649661,0.829939,0.599279,-0.322659,0.700588,-0.581794,-0.332373
4,1.749016,1.344857,-0.254866,0.649661,0.556688,0.599279,-0.322659,0.700588,-0.581794,-0.332373
5,1.741140,1.344857,-0.254866,0.649661,-0.263065,-0.941271,-0.322659,-0.503549,0.789196,1.355991
6,1.741140,1.344857,-0.254866,0.649661,-0.536315,0.599279,-0.322659,0.700588,-0.581794,-0.332373
7,1.741140,1.344857,-0.254866,0.649661,-1.082817,-0.941271,-0.322659,0.700588,-0.581794,-0.332373
8,1.741140,1.344857,-0.254866,0.649661,-2.175820,-0.941271,-0.322659,-0.770530,1.093171,1.355991
9,1.741140,1.344857,-0.254866,0.649661,-2.175820,0.599279,-0.322659,0.700588,-0.581794,-0.332373


In [56]:
import xgboost
xgboost.__version__

'0.81'

In [157]:
X = X.drop(['Day'], axis=1)
X

,Followers at Posting,Year,Month,Hour,Type,neg,neu,pos,compound
0,36984682,2019,5,0,1,0.000,1.000,0.000,0.0000
1,36984682,2019,5,23,1,0.000,1.000,0.000,0.0000
2,36984682,2019,5,21,2,0.000,0.711,0.289,0.5027
3,36955156,2019,5,18,2,0.000,1.000,0.000,0.0000
4,36955156,2019,5,15,2,0.000,1.000,0.000,0.0000
5,36955156,2019,5,2,1,0.000,1.000,0.000,0.0000
6,36955156,2019,5,1,2,0.000,0.556,0.444,0.6114
7,36955156,2019,5,22,2,0.000,1.000,0.000,0.0000
8,36955156,2019,5,22,2,0.171,0.829,0.000,-0.4389
9,36928789,2019,5,15,2,0.000,1.000,0.000,0.0000


In [161]:
y_pred = xgb.predict(X)
y_pred
train_df['Engagements'] = y_pred
with open("holdout_set_bad.csv", "w") as f:
    train_df.to_csv(f)